Тема урока: вложенные функции, замыкания
Вложенные функции
Замыкания
Ключевое слово nonlocal
Атрибут __closure__
Аннотация. Урок посвящен вложенным функциям и замыканиям.

Вложенные функции

In [4]:
dir(__builtin__)

['ArithmeticError',
 'AssertionError',
 'AttributeError',
 'BaseException',
 'BaseExceptionGroup',
 'BlockingIOError',
 'BrokenPipeError',
 'BufferError',
 'BytesWarning',
 'ChildProcessError',
 'ConnectionAbortedError',
 'ConnectionError',
 'ConnectionRefusedError',
 'ConnectionResetError',
 'DeprecationWarning',
 'EOFError',
 'Ellipsis',
 'EncodingWarning',
 'EnvironmentError',
 'Exception',
 'ExceptionGroup',
 'False',
 'FileExistsError',
 'FileNotFoundError',
 'FloatingPointError',
 'FutureWarning',
 'GeneratorExit',
 'IOError',
 'ImportError',
 'ImportWarning',
 'IndentationError',
 'IndexError',
 'InterruptedError',
 'IsADirectoryError',
 'KeyError',
 'KeyboardInterrupt',
 'LookupError',
 'MemoryError',
 'ModuleNotFoundError',
 'NameError',
 'None',
 'NotADirectoryError',
 'NotImplemented',
 'NotImplementedError',
 'OSError',
 'OverflowError',
 'PendingDeprecationWarning',
 'PermissionError',
 'ProcessLookupError',
 'RecursionError',
 'ReferenceError',
 'ResourceWarning',
 'Runti

In [5]:
def power_factory(exp):
    def power(base):
        return base ** exp

    return power


square = power_factory(2)
square(10)

100

Функция power является замыканием (closure). Это значит, что она сохраняет ссылку на значение переменной exp из окружающей области видимости, когда она была создана. Даже после того как выполнение покидает область power_factory, функция power продолжает иметь доступ к значению exp, которое было передано при её создании.

In [7]:
def strip_string(strip_chars=' '):
    def do_strip(string):
        return string.strip(strip_chars)

    return do_strip


strip1 = strip_string()
strip2 = strip_string(' !&,.?;:')

print(strip1(' he:llo world ?!'))
print(strip2(' hello world ?!'))

he:llo world ?!
hello world


In [12]:
def mean():
    total = 0
    length = 0

    def _mean(number):
        nonlocal total, length
        total += number
        length += 1
        return total / length

    return _mean


current_mean = mean()
current_mean(10)
current_mean(15)
current_mean(11)
current_mean(13)

12.25

In [29]:
g = globals().items()
for i, (k, v) in enumerate(g):
    print(i, k, v)
    if i > 5:
        break
print()
last_item = list(g)[-1]
print(last_item)

0 __name__ __main__
1 __doc__ Automatically created module for IPython interactive environment
2 __package__ None
3 __loader__ None
4 __spec__ None
5 __builtin__ <module 'builtins' (built-in)>
6 __builtins__ <module 'builtins' (built-in)>

('_i29', 'g = globals().items()\nfor i, (k, v) in enumerate(g):\n    print(i, k, v)\n    if i > 5:\n        break\nprint()\nlast_item = list(g)[-1]\nprint(last_item)')


In [33]:
def func(arg):
    var = 100
    print(locals())
    another = 200


func(300)

{'arg': 300, 'var': 100}


In [35]:
dir()[:10]

['In', 'Out', '_', '_1', '_10', '_11', '_12', '_13', '_16', '_2']

In [36]:
def func():
    var = 100
    print(dir())
    verr = 200


func()

['var']


Python позволяет определять функции внутри других функций. Их называют вложенными функциями или внутренними функциями.

In [37]:
def speak(text):
    def whisper(t):  # объявляем вложенную функцию
        return t.lower() + '...'

    return whisper(text)  # вызываем вложенную функцию и возвращаем ее результат


print(speak('Hello, World'))

hello, world...


Каждый раз, когда мы вызываем функцию speak(), она определяет новую внутреннюю функцию whisper(), а затем вызывает ее. При этом функция whisper() не существует вне родительской функции speak().

In [38]:
whisper('Hello')

NameError: name 'whisper' is not defined

In [39]:
speak.whisper('Hello')

AttributeError: 'function' object has no attribute 'whisper'

Если нам нужно получить доступ к вложенной функции whisper() извне, то мы можем вернуть ее (вложенную функцию) вызывающей стороне родительской функции.

Функция высшего порядка get_speak_func() определяет две вложенные функции whisper() и yell(). В зависимости от аргумента volume, переданного родительской функции get_speak_func(), она выбирает и возвращает вызывающей стороне одну из вложенных функций:

In [40]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'

    def yell(text):
        return text.upper() + '!'

    if volume > 0.5:
        return yell
    else:
        return whisper

Обратите внимание на то, что функция get_speak_func() на самом деле не вызывает одну из своих вложенных функций — она просто выбирает подходящую функцию на основе аргумента volume, а затем возвращает объект этой функции.

In [41]:
whisper = get_speak_func(0.3)  # функция whisper()
yell = get_speak_func(0.7)  # функция yell()

print(whisper('Hello'))  # говорим шепотом
print(yell('Hello'))  # кричим

hello...
HELLO!


Таким образом, функции не только могут принимать поведение через аргументы, но также могут возвращать поведение.

Замыкания

Вложенные функции могут захватывать и переносить с собой часть состояния родительской функции.

Перепишем немного функцию get_speak_func(), чтобы новая версия сразу принимала два аргумента volume и text:

In [42]:
def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'

    def yell():
        return text.upper() + '!'

    if volume > 0.5:
        return yell
    else:
        return whisper

In [43]:
yell = get_speak_func('Hello, World', 0.7)

print(yell())

HELLO, WORLD!


Теперь вложенные функции whisper() и yell() не имеют параметра text. Они его получают и используют через родительскую функцию get_speak_func(). Функции, которые делают это, называются замыканиями. Замыкание запоминает значения из включающей его области, даже если поток программы больше не находится в этой области.

Таким образом, замыкание — это особый вид функции. Она определена в теле другой функции и создаётся каждый раз во время её выполнения. Синтаксически это выглядит как функция, находящаяся целиком в теле другой функции. При этом вложенная функция содержит ссылки на локальные переменные внешней функции. Каждый раз при выполнении внешней функции происходит создание нового экземпляра внутренней функции, с новыми ссылками на переменные внешней функции.

In [44]:
def closure():
    count = 0

    def inner():
        nonlocal count
        count += 1
        print(count)

    return inner


start = closure()
another = closure()  # другое замыкание, со своими локальными значениями

start()  # выводит 1
start()  # выводит 2

another()  # выводит 1

start()  # выводит 3

1
2
1
3


Замыкания очень полезны при решении многих задач. С их помощью функции могут не только возвращать поведение, но и предварительно настраивать это поведение.

Пример 1. В этом примере функция greeting_creator() служит фабрикой для создания и настройки функции приветствия. Обратите внимание на то, что вложенная функция greet() может обращаться к аргументу greeting_word своей родительской функции greeting_creator()

In [45]:
def greeting_creator(greeting_word):
    def greet(name):
        return f'{greeting_word}, {name}'

    return greet

In [46]:
say_hi = greeting_creator('Hi')
say_hello = greeting_creator('Hello')

print(say_hi('Timur'))
print(say_hello('Soslan'))

Hi, Timur
Hello, Soslan


Пример 2. В этом примере функции make_adder() и multiplier_of() служат фабриками для создания и настройки функций «сумматора» и «мультипликатора». Обратите внимание на то, что вложенные функции add() и mult() могут обращаться к аргументу n своих родительских функций make_adder() и multiplier_of().

In [47]:
def make_adder(n):
    def add(x):
        return x + n

    return add


def multiplier_of(n):
    def mult(x):
        return x * n

    return mult

In [48]:
plus_3 = make_adder(3)
plus_5 = make_adder(5)
multiply_3 = multiplier_of(3)
multiply_5 = multiplier_of(5)

print(plus_3(10), plus_3(100))
print(plus_5(10), plus_5(100))
print(multiply_3(10), multiply_3(100))
print(multiply_5(10), multiply_5(100))

13 103
15 105
30 300
50 500


Имейте в виду, что вложенные функции, которые возвращает родительская функция, могут быть анонимными. Приведенный выше код можно переписать в виде:

In [49]:
def make_adder(n):
    return lambda x: x + n


def multiplier_of(n):
    return lambda x: x * n

Пример 3. В этом примере функция line_generator() служит фабрикой для создания и настройки линейной функции вида y=kx+b. Обратите внимание на то, что вложенная функция func() может обращаться к аргументам k и b своей родительской функции line_generator().

In [50]:
def line_generator(k, b):
    def func(x):
        return k * x + b

    return func

In [51]:
line_func_1 = line_generator(2, 5)  # получаем функцию y = 2*x + 5
line_func_2 = line_generator(-6, 9)  # получаем функцию y = -6*x + 9

print(line_func_1(10))  # печатаем значение 2*10 + 5
print(line_func_2(4))  # печатаем значение -6*4 + 9

25
-15


Обратите внимание на то, что во всех предыдущих примерах наши вложенные функции (замыкания) использовали нелокальные переменные, которые являлись параметрическими для родительской функции. Вложенные функции могут обращаться ко всем локальным переменным родительской функции, независимо от того, были они определены во внешней функции или пришли из аргументов внешней функции.

Параметрические переменные тоже считаются локальными переменными.

In [52]:
def f(x):
    z = 2

    def g(y):
        return z * x + y  # обращение к нелокальной переменной z и параметрической переменной x

    return g


h = f(5)
print(h(1))

11


Нелокальные переменные

При поиске переменной с указанным именем приоритет (правило разрешения имен) следующий:

сначала ищем локальную переменную
если не нашли локальную, ищем нелокальную
если не нашли нелокальную, ищем глобальную
и в самом конце — встроенную в язык
Промежуточная область видимости, которая доступна вложенным функциям, называется нелокальной. Если вложенность функций больше двух уровней, нелокальная переменная ищет в «ближайшей» области видимости, т. е. в функции вложенностью на один меньше. Если не находит, поиск переходит в самую ближнюю из внешних областей видимости, затем в чуть более далекую — и так далее, пока не найдется нужное имя. Фактически интерпретатор ищет там, «где поближе».

Ключевое слово nonlocal

Как мы уже знаем, вложенная функция видит переменные в родительской функции, которые называются нелокальными или свободными переменными.

In [54]:
def outer_function():
    num = 5

    def inner_function():  # определяем вложенную функцию
        num += 10
        print(num)

    inner_function()  # вызываем вложенную функцию


outer_function()

UnboundLocalError: cannot access local variable 'num' where it is not associated with a value

Дело в том, что вложенная функция не просто обращается (получает значение) к переменной num, но и пытается его изменить! Внутренняя функция видит переменные в объемлющей функции, но, если она хочет такую переменную изменить, должна объявить ее nonlocal

In [55]:
def outer_function():
    num = 5

    def inner_function():  # определяем вложенную функцию
        nonlocal num
        num += 10
        print(num)

    inner_function()  # вызываем вложенную функцию


outer_function()

15


Атрибут __closure__

Все функции содержат специальный атрибут __closure__, который представляет из себя кортеж, содержащий данные, связанные с вложенными областями видимости, то есть с нелокальными переменными.

In [69]:
def outer_function(arg):
    num = 5
    name = 'Timur'
    numbers = [1, 2, 3]

    # Сохраняем локальные переменные в словарь
    outer_locals = locals()

    # print("Локальные переменные в outer_function:", outer_locals)

    def inner_function():  # определяем вложенную функцию
        print(arg)
        print(num)
        print(numbers)

    return inner_function, outer_locals  # Возвращаем и функцию, и словарь локальных переменных


# Вызов outer_function
inner, outer_locals = outer_function('python')

for var in inner.__closure__:
    print(var.cell_contents)

# Печать сохранённых локальных переменных
print("Словарь локальных переменных из outer_function:", outer_locals)

python
5
[1, 2, 3]
Словарь локальных переменных из outer_function: {'arg': 'python', 'name': 'Timur', 'num': 5, 'numbers': [1, 2, 3]}


Обратите внимание на то, что кортеж __closure__ содержит внутри себя специальный тип данных. Для получения самого значения захваченной переменной нужно использовать атрибут cell_contents.

Когда использовать замыкания

Концепция замыканий, то есть функций, захватывающих нелокальные переменные, находят много применений при написании кода. Замыкания хороши для:

воздержания от жестко закодированных констант
воздержания от использования глобальных переменных
воздержания от создания ненужных типов данных (классов)
замыкания нужны для реализации декораторов, о которых мы поговорим чуть позже
Сокрытие данных основное преимущество замыканий.

Примечания

https://realpython.com/python-scope-legb-rule/#using-enclosing-scopes-as-closures

Примечание 3. Вложенные (внутренние) функции позволяют избавиться от глобальных переменных. Такая техника позволяет сделать внешнюю для функции переменную, но при этом спрятанную от посторонних глаз, в отличие от глобальной. Такие переменные нужны в первую очередь для того, чтобы хранить какие-то данные, относящиеся к функции, между вызовами функции. Локальные переменные стираются при выходе из функции, глобальные — сохраняются, но видны всему свету, а нелокальные — идеальное сочетание закрытости и «сохраняемости».

Примечание 4. Если функция использует глобальные переменные, это тоже замыкание. Но чаще всего замыканием называют все-таки функцию, которая использует нелокальные переменные. Такая функция как бы «таскает за собой» свои внешние переменные, но никому их не показывает.

In [70]:
def outer(x):
    y = 5
    z = 10

    def inner():
        nonlocal y
        y += 1
        z = 20
        print('x =', x)
        print('z =', z)

    inner()
    print('x =', x)
    print('y =', y)
    print('z =', z)


outer(5)

x = 5
z = 20
x = 5
y = 6
z = 10


In [71]:
def outer(x):
    y = 20

    def inner(z):
        t = 30
        return x + y + z + t

    return inner


func = outer(10)

for var in func.__closure__:
    print(var.cell_contents)

10
20


In [72]:
def outer(x):
    def inner():
        return x

    x = None
    return inner()


print(outer(10))

None


In [73]:
def outer(x):
    def inner():
        return x

    x = None
    return inner


print(outer(10)())

None


In [74]:
def make_counter(i):
    def counter():
        nonlocal i
        i += 1
        return i

    return counter


counter1 = make_counter(100)
counter2 = make_counter(200)

print(counter1(), counter1())
print(counter2(), counter2())

101 102
201 202


In [75]:
def greeting(language):
    def greeting_ru():
        print('Привет!')

    def greeting_fr():
        print('Bonjour!')

    if language == 'ru':
        return greeting_ru
    if language == 'fr':
        return greeting_fr


func = greeting('fr')
func()

Bonjour!


Функция power()
Реализуйте функцию power(), которая принимает один аргумент:

degree — целое число
Функция power() должна возвращать функцию, которая принимает в качестве аргумента целое число x и возвращает значение x в степени degree.

Примечание 1. Рассмотрим пример из первого теста. Вызов power(2) возвращает функцию, которая принимает в качестве аргумента число и возводит его во вторую степень. Функция присваивается переменной square. Далее полученная функция вызывается с аргументом 5 и возвращает значение 5 в степени 2 =25.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимую функцию power(), но не код, вызывающий ее.

In [80]:
def power(degree):
    def inner(number):
        return number ** degree

    return inner


square = power(2)
print(square(5))

25


In [ ]:
def power(degree):
    return lambda x: x ** degree

Функция generator_square_polynom()
Рассмотрим семейство функций — квадратных трехчленов. Все эти функции имеют один и тот же вид:f(x)=ax 2 +bx+c
Реализуйте функцию generator_square_polynom(), которая принимает три аргумента в следующем порядке:
a — вещественное число, коэффициент 
b — вещественное число, коэффициент 
c — вещественное число, коэффициент 
Функция generator_square_polynom() должна возвращать функцию, которая принимает в качестве аргумента вещественное число x и возвращает значение выражения 
ax 2 +bx+c.

Примечание 1. Рассмотрим пример из первого теста. Вызов generator_square_polynom(1, 2, 1) возвращает функцию, соответствующую квадратному трехчлену
x 2 +2x+1.  Функция присваивается переменной f. Далее полученная функция вызывается с аргументом 5 и возвращает значение 
5 2 +5⋅2+1=36.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимую функцию generator_square_polynom(), но не код, вызывающий ее.

In [83]:
def generator_square_polynom(a, b, c):
    def inner_function(x):
        return a * x ** 2 + b * x + c

    return inner_function


f = generator_square_polynom(1, 2, 1)
print(f(5))

print(generator_square_polynom(9, 52, 64)(8))

f = generator_square_polynom(26, 83, 22)
print(f(55))

36
1056
83237


Функция sourcetemplate()
Строка запроса (query string) — часть URL адреса, содержащая ключи и их значения. Она начинается после вопросительного знака и идет до конца адреса. Например:

https://beegeek.ru?name=timur     # строка запроса: name=timur
Если параметров в строке запроса несколько, то они отделяются символом амперсанда &:

https://beegeek.ru?name=timur&color=green     # строка запроса: name=timur&color=green 
Реализуйте функцию sourcetemplate(), которая принимает один аргумент:

url — URL адрес
Функция sourcetemplate() должна возвращать функцию, которая принимает произвольное количество именованных аргументов и возвращает url адрес, объединенный со строкой запроса, сформированной из переданных аргументов. При вызове без аргументов она должна возвращать исходный url адрес без изменений.

Примечание 1. Параметры в строке запроса должны располагаться в лексикографическом порядке ключей.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимую функцию sourcetemplate(), но не код, вызывающий ее.

In [95]:
def sourcetemplate(url: str):
    def inner_function(**kwargs):
        if len(kwargs) == 0:
            return url
        query_params = [f'{k}={v}' for k, v in sorted(kwargs.items())]
        return f'{url}?{'&'.join(query_params)}'

    return inner_function


url = 'https://beegeek.ru'
load = sourcetemplate(url)
print(load(name='timur'))

url = 'https://stepik.org/lesson/651459/step/14'
load = sourcetemplate(url)
print(load(thread='solutions', unit=648165))

url = 'https://beegeek.ru'
load = sourcetemplate(url)
print(load())

url = 'https://all_for_comfort_life.com'
load = sourcetemplate(url)
print(load(smartphone='iPhone', notebook='huawei', sale=True))

https://beegeek.ru?name=timur
https://stepik.org/lesson/651459/step/14?thread=solutions&unit=648165
https://beegeek.ru
https://all_for_comfort_life.com?notebook=huawei&sale=True&smartphone=iPhone


Функция date_formatter()
Нередко в разных странах используются разные форматы дат. Рассмотрим часть из них:

код страны	формат даты
ru	DD.MM.YYYY
us	MM-DD-YYYY
ca	YYYY-MM-DD
br	DD/MM/YYYY
fr	DD.MM.YYYY
pt	DD-MM-YYYY
Реализуйте функцию date_formatter(), которая принимает один аргумент:

country_code — код страны
Функция date_formatter() должна возвращать функцию, которая принимает в качестве аргумента дату (тип date) и возвращает строку с данной датой в формате страны с кодом country_code.

Примечание 1. Гарантируется, что в функцию date_formatter() передаются только те коды стран, что перечислены в приведенной выше таблице.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимую функцию date_formatter(), но не код, вызывающий ее

In [8]:
from datetime import date

GLOBAL_DIC = {'ru': '%d.%m.%Y',
              'us': '%m-%d-%Y',
              'ca': '%Y-%m-%d',
              'br': '%d/%m/%Y',
              'fr': '%d.%m.%Y',
              'pt': '%d-%m-%Y', }


def date_formatter(country_code: str):
    def inner(arg):
        return date.strftime(arg, GLOBAL_DIC.get(country_code))

    return inner


date_ru = date_formatter('ru')
today = date(2022, 1, 25)
print(date_ru(today))

date_ru = date_formatter('us')
today = date(2025, 1, 5)
print(date_ru(today))

date_ru = date_formatter('ca')
today = date(2015, 12, 7)
print(date_ru(today))

25.01.2022
01-05-2025
2015-12-07


In [ ]:
from datetime import date

def date_formatter(country_code):
    def date_form(data):
        d = {'ru': '%d.%m.%Y', 'us': '%m-%d-%Y',
             'ca': '%Y-%m-%d', 'br': '%d/%m/%Y',
             'fr': '%d.%m.%Y', 'pt': '%d-%m-%Y', }
        return date.strftime(data, d[country_code])
    return date_form

Функция sort_priority() 🌶️
Реализуйте функцию sort_priority(), которая принимает два аргумента в следующем порядке:

values — список чисел
group — список, кортеж или множество чисел
Функция должна сортировать по неубыванию список чисел values, делая при этом приоритетной группу чисел из group, которая должна следовать первой.

Примечание 1. В тестирующую систему сдайте программу, содержащую только необходимую функцию sort_priority(), но не код, вызывающий ее.

In [52]:
def sort_priority(values: list, group):
    def inner():
        nonlocal values
        nonlocal group
        sorted_group = sorted([v for v in values if v in group])
        sorted_others = sorted([v for v in values if v not in group])
        values[:] = sorted_group + sorted_others
        return values
    inner()


numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {5, 7, 2, 3}
sort_priority(numbers, group)

print(numbers)

numbers = [150, 200, 300, 1000, 50, 20000]
sort_priority(numbers, [300, 100, 200])

print(numbers)

numbers = [9, 8, 7, 6, 5, 4, 3, 2, 1]
sort_priority(numbers, (300, 100, 200))

print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]
[200, 300, 50, 150, 1000, 20000]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
def sort_priority(numbers, group):
    numbers.sort(key=lambda x: (x not in group, x))

In [ ]:
def sort_priority(values, group):
    first = set(group) & set(values)
    second = set(values) - set(group)
    values[:] = sorted(first) + sorted(second)

In [ ]:
def sort_priority(values, group):
    new_group = set(values).intersection(set(group))
    new_values = set(values).difference(new_group)
    values[:] = sorted(list(new_group)) + sorted(list(new_values))